In [ ]:
!nvidia-smi
!nvcc --version

Mon Oct 31 09:08:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# nerf-pl 훈련하기

원격으로 Nerf pl을 훈련하고, Data를 보관하기 위해 colab 을 별도로 만들었다.

이를 위해 최신사양에 맞는 colab에서 돌아가게 수정한 nerf-pl 을 사용한다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TARGET_PATH = "/content/drive/MyDrive/YonseiUniversity/MetaArt"
%cd $TARGET_PATH

/content/drive/MyDrive/YonseiUniversity/MetaArt


Target path 에 nerf-pl 폴더가 없는 경우 github 에서 clone 한다.

만약에 있는 경우는 아래 단계는 스킵한다.

In [ ]:
!git clone https://github.com/maroomir/nerf_pl.git

fatal: destination path 'nerf_pl' already exists and is not an empty directory.


pip 설치를 위해 여기는 반드시 실행해야한다.

In [ ]:
%cd nerf_pl
!pip install -r requirements.txt

/content/drive/MyDrive/YonseiUniversity/MetaArt/nerf_pl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## LLFF input

Data 위치를 입력한다.

Data 위치는 sample 도 괜찮고, 아니면 우리가 직접 만든 LLFF Data 도 괜찮다.

단, 무조건 LLFF 형태여야 한다.

In [ ]:
import os
os.environ['DATA_DIR'] = "/content/drive/MyDrive/YonseiUniversity/MetaArt/nerf-pytorch/data/nerf_llff_data/fortress"
os.environ['IMG_W'] = "504" # image width (do not set too large)
os.environ['IMG_H'] = "378" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "30" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "fortress" # name of the experience (arbitrary)

colmap 에 정의된 pose 수와 Image 수가 반드시 맞아야한다.

In [ ]:
import numpy as np
import glob
poses_bounds = np.load(os.path.join(os.environ['DATA_DIR'], 'poses_bounds.npy'))
colmap_cnt = len(poses_bounds)
img_cnt = len(glob.glob(os.path.join(os.environ['DATA_DIR'], 'images/*')))
print(f"COLMAP={colmap_cnt} - IMAGES={img_cnt}")

COLMAP=42 - IMAGES=42


## Training

colmap 갯수 확인에 이상이 없는 경우, 돌릴 수 있다.

In [ ]:
!python train.py \
   --dataset_name llff \
   --root_dir "$DATA_DIR" \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/test_tube.py:106: LightningDeprecationWarning: The TestTubeLogger is deprecated since v1.5 and will be removed in v1.7. We recommend switching to the `pytorch_lightning.loggers.TensorBoardLogger` as an alternative.
  "The TestTubeLogger is deprecated since v1.5 and will be removed in v1.7. We recommend switching to the"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f27a2eb0310>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f27a2eb0310>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/train

## Test

결과는 results/llff/$SCENE 에 저장된다.

In [ ]:
os.environ['SCENE'] = 'fortress'
os.environ['CKPT_PATH'] = '/content/drive/MyDrive/YonseiUniversity/MetaArt/nerf_pl/logs/fortress/version_None/checkpoints/epoch=29-step=228840.ckpt'

!python eval.py \
   --root_dir "$DATA_DIR" \
   --dataset_name llff --scene_name $SCENE \
   --img_wh $IMG_W $IMG_H --N_importance 64 --ckpt_path $CKPT_PATH

100% 120/120 [08:17<00:00,  4.14s/it]
